# Db2 v12.1.2でベクトルデータが入るようになったので、langchainから使ってみた
https://qiita.com/nishikyon/items/c84db0f885dcf9609276

# 1. Excelをベクトル化してlangchainでDb2に入れよう！

### 1. 必要なライブラリーのインストール
(環境によって他に不足ライブラリがあれば適時インストールしてください）

In [ ]:
# 既にインストール済みであればはこのコードの実行は不要です
!pip install langchain-db2
!pip install ibm_db
!pip install ibm_db_sa sqlalchemy
!pip install langchain-huggingface
!pip install sentence_transformers
!pip install tqdm

### 2. Db2接続情報の設定とDb2への接続
`<XXXX>`を接続先のDb2環境の値に設定してください。
SSL接続の場合はdsn内のコメントを外してください。

In [ ]:
import ibm_db
import ibm_db_dbi

database = "<database名>"
username = "<userid>"
password = "<password>"
hostname = "<hostname>"
port = "<port番号>"

dsn = (
    f"DATABASE={database};"
    f"HOSTNAME={hostname};"
    f"PORT={port};"
    f"PROTOCOL=TCPIP;"
    f"UID={username};"
    f"PWD={password};"
    # "SECURITY=SSL;"  # SSL接続の場合はコメントをはずす
)

# Db2への接続
try:
    # Connect using DSN string
    conn = ibm_db.connect(dsn, "", "")
    connection = ibm_db_dbi.Connection(conn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!", e)

### 3. 必要ライブラリーのImport

In [ ]:
import pandas as pd
import json

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document

from langchain_db2 import db2vs
from langchain_db2.db2vs import DB2VS

### 4. ベクトル化するデータの作成
#### 4-1. Excelデータの取得 

In [ ]:
!wget https://github.com/IBM/japan-technology/raw/refs/heads/main/techxchange/2024-watsonx-handson-1/data/TechXchangeJapan2024.xlsx -O TechXchangeJapan2024.xlsx

#### 4-2. Excelファイルの内容を pandas.DataFrameに読み込む

In [ ]:
path="./"
filename='TechXchangeJapan2024.xlsx'
excel_file = path+filename

df_list = []

# 全てのシートを読み込み、リストdf_listに格納
for sheet_name in pd.ExcelFile(excel_file).sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)

    # Db2のlangchainではidは小文字が必須のため、ここで小文字変換しておく
    df.rename(columns={'ID': 'id'}, inplace=True)
    
    df_list.append(df)
    print(f"\nExcelシート名: {sheet_name}")
    display(df.head()) #各シート最初の5　行　表示

#### 4-3. 行をJSON化し、metadataとして`Category`,`id`を抜き出す

In [ ]:
import numpy as np
json_doc_list=[]
json_meta_list=[]

for df in df_list:
    # 各シートのデータフレームに対する処理
    
    # 行をJSONフォーマットに変換
    json_doc_string = json.loads(df.to_json(orient='records', force_ascii=False))
   
    # metaデータとして'Category','id'を抜き出し, JSONに変換
    json_meta_string =  json.loads(df[['Category','id']].to_json(orient='records', force_ascii=False), parse_int=str)

    # 各シートのJSON Listを1つのListに結合
    json_doc_list.extend(json_doc_string)
    
    json_meta_list.extend( json_meta_string)

#中身確認 最初の5行
print("ベクトル化するデータ 最初の5行")
for index, item in enumerate(json_doc_list[0:5]):
    print(index + 1, item)

print("\nmetaデータ 最初の5行")
for index, item in enumerate(json_meta_list[0:5]):
    print(index + 1, item)

#### 4-4. 1行の情報をlangchainのDocumentにし、Listを作成

In [ ]:
# DocumentのListをjson_doc_listとjson_meta_listから作成
# page_contentはjson_doc_listの一行分のjson
# metadataはjson_meta_listのの一行分のjson

# 以下と同じコード
# docs = []
# for doc_str, meta_str in zip(json_doc_list, json_meta_list):
#     docs.append(Document(page_content=json.dumps(doc_str, ensure_ascii=False), metadata=meta_str)) 
    
docs = [Document(page_content=json.dumps(doc_str, ensure_ascii=False), metadata=meta_str) 
        for doc_str, meta_str in zip(json_doc_list, json_meta_list)]

#中身確認 最初の二行
print(docs[0], "\n\n")

print(docs[1], "\n\n")

### 5. Embeddingモデルを作成

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

### 6. Db2にデータの挿入

In [ ]:
# vectest.techxchange_line_data に　データの挿入
# 同じ名前のテーブルがあった場合、内容は上書きされる
vector_store = DB2VS.from_documents(
    docs,
    embeddings,
    client=connection,
    table_name="vectest.techxchange_line_data",
    distance_strategy=DistanceStrategy.COSINE,
)

上記はベクトル化＋データの挿入を実施していますが、もし既にデータの入っているDb2にテーブルを指定し接続するのみの場合は以下のように接続してください：

In [ ]:
# 既存データを使う場合はこちらを実行
vector_store = DB2VS(
    embeddings,
    table_name="nishito.techxchange_line_data",
    client=connection,
    distance_strategy=DistanceStrategy.COSINE,
)

### 7. データの追加・削除
#### 7.1. データの追加　add_texts()使用

In [ ]:
texts = [
    '{"Key": "テスト1", "概要": "テスト用の概要1です。意味はないです", "id": "概要テスト1", "Category": "概要"}',
    '{"Key": "テスト2", "概要": "テスト用の概要2です。意味はないです", "id": "概要テスト2", "Category": "概要"}'
]
         
metadata = [
    { "id": "概要テスト1", "Category": "概要"},
    { "id": "概要テスト2", "Category": "概要"},
]

vector_store.add_texts(texts, metadata)

#### 7.2 データの追加　add_documents()使用

In [ ]:
texts = [
    {"Key": "テスト1doc", "概要": "テスト用の概要1です。意味はないです", "id": "概要テスト1", "Category": "概要"},
    {"Key": "テスト2doc", "概要": "テスト用の概要2です。意味はないです", "id": "概要テスト2", "Category": "概要"}
]
         
metadata = [
    { "id": "概要テスト1doc", "Category": "概要"},
    { "id": "概要テスト2doc", "Category": "概要"},
]

docs_add = [Document(page_content=json.dumps(doc_str, ensure_ascii=False), metadata=meta_str) 
        for doc_str, meta_str in zip(texts, metadata)]

vector_store.add_documents(docs_add)

#### 7.3 データの削除 delete()

In [ ]:
vector_store.delete(['概要テスト1', '概要テスト2', '概要テスト1doc', '概要テスト2doc'])

### 8. 挿入データの確認

In [ ]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# DB2の接続情報を設定
password_q = quote_plus(password)

# SQLAlchemyのエンジンを作成
url = f"ibm_db_sa://{username}:{password_q}@{hostname}:{port}/{database};SECURITY=SSL;"
engine = create_engine(url)

sql_str = """
SELECT ID, TEXT,
SYSTOOLS.BSON2JSON(METADATA) METADATA, 
VECTOR_SERIALIZE(EMBEDDING) EMBEDDING
FROM VECTEST.TECHXCHANGE_LINE_DATA
"""

# pandas.read_sql() を使用してpandas Dataframe型のdf_vecにSELECTの結果を入れる
df_vec = pd.read_sql(sql_str, engine)

# notebookの場合は以下で表示できます
df_vec

# 2. langchainでDb2にいれたデータを類似検索してみよう！

1に引き続き実行する場合は既にvector_storeは取得済みです。「2-1. similarity_search: オプションなし デフォルト」に進んでください。

ここから新たに実行する場合は以下でvector_storeを取得してください:
- connectionは「2. Db2接続情報の設定とDb2への接続」の方法で事前に作成してください。
その後以下のコードでlangchainのvector_storeを作成します。

In [ ]:
# ここから新たに実行する場合のみ実行する
vector_store = DB2VS(
    embeddings,
    table_name="vectest.techxchange_line_data",
    client=connection,
    distance_strategy=DistanceStrategy.COSINE,
)

### 2-1. similarity_search: オプションなし デフォルト

In [ ]:
# オプションなし
query = "IBM TechXchange Japanとは?"
docs = vector_store.similarity_search(query)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

### 2-2. similarity_search: 結果の取得数をkで指定(デフォルトは4)

In [ ]:
# 結果の取得数をkで指定(デフォルトは4)

docs = vector_store.similarity_search(query, k=10)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

### 2-3. similarity_search_with_score: スコアも一緒に出力

In [ ]:
# スコアも一緒に出してみます
# スコアはコサイン距離の場合は、0に近いほど類似度が高いです。

docs = vector_store.similarity_search_with_score(query, k=10)
for doc, score in docs:
    print({"score": score, "content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

### 2-4. similarity_search_with_score filterオプション: 'Category': '概要'　でフィルター

In [ ]:
# 'Category': '概要'　でフィルターしてみます
filter_criteria = {"Category": ["概要"]}
docs = vector_store.similarity_search_with_score(query, k=10, filter=filter_criteria)
for doc, score in docs:
    print({"score": score, "content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")